# transforms

> 2D Matrix transformations to manipulate a series of `(x,y)` coordinates.

In [ ]:
#| default_exp transforms

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
# |export
from dataclasses import dataclass

import numpy as np

## 2D Affine Transformations

In [ ]:
#| export
def apply_transform(coords_2d, xform):
    assert coords_2d.shape[1] == 2
    assert xform.shape[0] == 3
    assert xform.shape[1] == 3

    coords_full = np.concatenate([coords_2d, np.ones((coords_2d.shape[0], 1))], axis=1)
    assert coords_full.shape[0] == coords_2d.shape[0]
    assert coords_full.shape[1] == 3

    return xform.dot(coords_full.transpose()).transpose()

In [ ]:
#| export
def identity_xform():
    return np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])

In [ ]:
#| export
def scale_xform(scale_x, scale_y):
    return np.array([[scale_x, 0, 0], [0, scale_y, 0], [0, 0, 1]])

In [ ]:
#| export
def translate_xform(translate_x, translate_y):
    return np.array([[1, 0, translate_x], [0, 1, translate_y], [0, 0, 1]])

In [ ]:
#| export
def rotate_xform(rotate_angle):
    if rotate_angle % 360 == 0:
        return identity_xform()
    theta = np.radians(rotate_angle)
    cos_theta = np.cos(theta)
    sin_theta = np.sin(theta)
    return np.array([[cos_theta, -sin_theta, 0], [sin_theta, cos_theta, 0], [0, 0, 1]])

## Bounding Boxes

In [ ]:
#| export
class BoundingBox:
    xmin: float
    xmax: float
    ymin: float
    ymax: float
    xrange: float
    yrange: float

    def __init__(self, xmin, xmax, ymin, ymax):
        assert xmin < xmax
        assert ymin < ymax
        self.xmin = xmin
        self.xmax = xmax
        self.ymin = ymin
        self.ymax = ymax
        self.xrange = xmax - xmin
        self.yrange = ymax - ymin

    @staticmethod
    def create(coords: np.ndarray):
        # rank-2
        if len(coords.shape) == 2:
            xmin = coords[:, 0].min()
            xmax = coords[:, 0].max()

            ymin = coords[:, 1].min()
            ymax = coords[:, 1].max()

            return BoundingBox(
                xmin=xmin, xmax=xmax, ymin=ymin, ymax=ymax
            )
        else:
            raise Exception(f"invalid coordinates passed - expected rank-2 matrix but got rank-{len(coords.shape)}")

    def area(self):
        return self.xrange * self.yrange

    def intersection(self, other):
        bb1 = self
        bb2 = other

        assert bb1.xmin < bb1.xmax
        assert bb1.ymin < bb1.ymax
        assert bb2.xmin < bb2.xmax
        assert bb2.ymin < bb2.ymax

        x_left = max(bb1.xmin, bb2.xmin)
        y_top = max(bb1.ymin, bb2.ymin)
        x_right = min(bb1.xmax, bb2.xmax)
        y_bottom = min(bb1.ymax, bb2.ymax)

        if x_right < x_left or y_bottom < y_top:
            return None
        
        return BoundingBox(xmin=x_left, ymin=y_top, xmax=x_right, ymax=y_bottom, xrange=x_right-x_left, yrange=y_bottom-y_top)

    def iou(self, other):
        """
        Intersection over union - area of the overlap relative to combined area of the bounding boxes
        """
        overlap = self.intersection(other)
        if overlap:
            return overlap.area() / float(self.area() + other.area() - overlap.area())
        else:
            return None
        
    def normalization_xform(self, scale=1.0):
        """
        Produce a normalization transform - a set of transformations,
        given the input coordinates, to convert all coords into the range (0,1)
        """
        max_range = self.xrange if self.xrange > self.yrange else self.yrange
        return scale_xform(scale / max_range, scale / max_range).dot(
            translate_xform(-self.xmin, -self.ymin)
        )
    
    

: 

## Stroke-3 Conversion

In [ ]:
#| export
def strokes_to_points(strokes):
    all = []
    for s in strokes:
        eoc_col = np.zeros((s.shape[0], 1))
        eoc_col[-1, 0] = 1
        all.append(np.concatenate([s[:, :2], eoc_col], axis=1))
    return np.vstack(all)


def points_to_deltas(points):
    p2 = points.copy()
    # first row should stay the same
    # cols 0,1 of every row onwards should be a delta from the previous row.
    p2[1:, :2] = points[1:, :2] - points[:-1, :2]
    return p2


def strokes_to_deltas(strokes):
    points = strokes_to_points(strokes)
    return points_to_deltas(points)

In [ ]:
#| export
def deltas_to_points(_seq):
    seq = np.zeros_like(_seq)
    seq[:, 0:2] = np.cumsum(_seq[:, 0:2], axis=0)
    seq[:, 2] = _seq[:, 2]
    return seq


def points_to_strokes(_seq):
    strokes = np.split(_seq, np.where(_seq[:, 2] > 0)[0] + 1)
    return [
        s for s in strokes if len(s) > 0
    ]  # split sometimes returns an empty array at the end


def deltas_to_strokes(_seq):
    points = deltas_to_points(_seq)
    return points_to_strokes(points)

## RDP

In [ ]:
#| export
from rdp import rdp


def rdp_strokes(strokes, epsilon=1.0):
    return [rdp(s, epsilon=epsilon) for s in strokes]

In [ ]:
#| export
def stroke_rdp_deltas(rescaled_strokes, epsilon=2.0):
    rdp_result = rdp_strokes(rescaled_strokes, epsilon)
    deltas = strokes_to_deltas(rdp_result)

    ## roundtrip / sanity check
    # _rdp_result = stroke3.deltas_to_strokes(deltas)
    # default.plot_strokes(_rdp_result)

    return deltas

In [ ]:
#| hide
import nbdev

nbdev.nbdev_export()